<a href="https://colab.research.google.com/github/SumeetChougule/PM-HR/blob/main/notebooks/CNN_trial_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/DifferentiableUniverseInitiative/flowpm.git

In [ ]:
!pip install git+https://github.com/DifferentiableUniverseInitiative/flowpm.git

In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import tensorflow.keras.layers as tfl
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.compat.v1.Session(config=config)
from tensorflow.keras import datasets, layers, models, losses

import tensorflow_probability as tfp
tfd = tfp.distributions

# Input data from PM



In [ ]:
import flowpm
from astropy.cosmology import Planck15
from flowpm import linear_field, lpt_init, nbody, cic_paint, cic_readout
from flowpm.utils import r2c3d, c2r3d

from scipy.interpolate import InterpolatedUnivariateSpline as iuspline


In [ ]:
bs, nc = 100, 32
nsteps = 5
a0, af, nsteps = 0.1, 1.0,  nsteps
stages = np.linspace(a0, af, nsteps, endpoint=True)
donbody = False
dnoise = 1. #0.1

klin, plin = np.loadtxt('flowpm/flowpm/data/Planck15_a1p00.txt').T
ipklin = iuspline(klin, plin)

In [ ]:
@tf.function
def pm(linear):
    print("PM graph")
    cosmo = flowpm.cosmology.Planck15()
    state = lpt_init(cosmo, linear, a=a0, order=2)
    final_state = nbody(cosmo, state,  stages, nc)
    tfinal_field = cic_paint(tf.zeros_like(linear), final_state[0])
    return final_state, tfinal_field

In [ ]:
ic = linear_field(nc, bs, ipklin, name='pm').numpy()
state, fin = pm(tf.constant(ic))
data = fin + np.random.normal(0, dnoise, nc**3).reshape(fin.shape)

data = data.numpy().astype(np.float32)

PM graph


In [ ]:
state.shape

TensorShape([3, 1, 32768, 3])

CNN $→$ CIC_readout $→$ Flatten $→ MLP $

In [ ]:
def cnn_model(input_shape):
    model = tf.keras.Sequential([
        tfl.InputLayer(input_shape= input_shape),
        tfl.Conv3D(filters = 32, kernel_size= 3, strides=(2,2,2), padding='same', activation= 'relu'),
        tfl.MaxPooling3D(pool_size=(2,2,2), strides = None, padding ='valid'),
        tfl.Conv3D(filters = 64, kernel_size= 3, strides=(2,2,2), padding='same', activation= 'relu'),
        tfl.MaxPooling3D(pool_size=(3,3,3), strides = None, padding ='valid')
        ])
    return model

cnn = cnn_model((nc,nc,nc,1))

In [ ]:
cnn_op=cnn(data).numpy().astype(np.float32)

In [ ]:
cnn_op.shape

(1, 1, 1, 1, 64)

In [ ]:
cnn.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv3d_16 (Conv3D)          (None, 16, 16, 16, 32)    896       
                                                                 
 max_pooling3d_12 (MaxPoolin  (None, 8, 8, 8, 32)      0         
 g3D)                                                            
                                                                 
 conv3d_17 (Conv3D)          (None, 4, 4, 4, 64)       55360     
                                                                 
 max_pooling3d_13 (MaxPoolin  (None, 1, 1, 1, 64)      0         
 g3D)                                                            
                                                                 
Total params: 56,256
Trainable params: 56,256
Non-trainable params: 0
_________________________________________________________________


Interpolation of CNN output

In [ ]:
mesh = tf.zeros([1, nc, nc, nc])
p_pos = cic_readout(  , state)
MLP_ip = tfl.Flatten()(p_pos)

Need to fix the shape out CNN output for the cic_readout.

In [ ]:
def MLP(x):
  model = tf.keras.Sequential([
      tfl.InputLayer(input_shape = meshf.shape),
      tfl.Dense(64, activation = 'relu'),
      tfl.Dense(32, activation = 'relu'),
# how many layers for the output?      
      tfl.Dense( x, activation = 'linear')
  ])
  return model 

f_op = MLP(x)

In [ ]:
f_op.summary()